 This dataset contains all transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

This notebook consists of the below:
- Data cleaning
- EDA
- Feature Engineering with RFM
- Outlier treatment with Isolation Forest
- KMeans clustering
- Visualization of clusters with multiple displays
- Selection of products to display to each customer segment as they traverse the web site:
    * Top 5 most purchased products per customer segment
    * For customers that purchased the top product in each customer segment, the top 3 most popular items are determined i.e. "Customers that purchased this also purchased..."


In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors as mcolors
from scipy.stats import linregress
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans
from tabulate import tabulate
from collections import Counter

%matplotlib inline

In [ ]:
# Initialize Plotly for use in the notebook
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
# Configure Seaborn plot styles: Set background color and use dark grid
sns.set(rc={'axes.facecolor': '#fcf0dc'}, style='darkgrid')

In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding="ISO-8859-1")

<div style="border-radius:10px; padding: 15px; background-color: #ffeacc; font-size:130%; text-align:left">

<h2 align="left"><font color=#ff6200>Dataset Description:</font></h2>

| __Variable__   | __Description__ |
|     :---       |       :---      |      
| __InvoiceNo__  | Code representing each unique transaction.  If this code starts with letter 'c', it indicates a cancellation. |
| __StockCode__  | Code uniquely assigned to each distinct product. |
| __Description__| Description of each product. |
| __Quantity__   | The number of units of a product in a transaction. |
| __InvoiceDate__| The date and time of the transaction. |
| __UnitPrice__  | The unit price of the product in sterling. |
| __CustomerID__ | Identifier uniquely assigned to each customer. |
| __Country__    | The country of the customer. |


In [ ]:
df.head(10)

In [ ]:
df.info()

Description and CustomerID has some missing values.

In [ ]:
# Summary statistics for numerical variables
df.describe()

There are 541K transactions in this data set. The negative Quantity and UnitPrice values will need further analysis for understanding, as negative numbers for these are unusual.

In [ ]:
# Summary statistics for categorical variables
df.describe(include='object')

Of the 541K transactions, there are 4,223 unique descriptions (or products) purchased from 38 countries, most frequently the United Kingdom.

### Handling missing values

In [ ]:
def perc_mv(x, y):
    perc = y.isnull().sum() / len(x) * 100
    return perc

print('Missing value ratios(%):\nDescription: {}\nCustomerID: {}'.format(perc_mv(df, df['Description']),
                                                                                   perc_mv(df, df['CustomerID'])
                                                                                   ))


The `CustomerID` column contains nearly a quarter of missing data. As clustering is based on customer behavior and preferences, it's crucial to have accurate data on customer identifiers. As such there is no viable Inputting strategy for this scenario. These rows will need to be dropped.
    
The `Description` column has a minor percentage of missing values, and can be safely removed.

In [ ]:
# Extracting rows with missing values in 'CustomerID' or 'Description' columns
df[df['CustomerID'].isnull() | df['Description'].isnull()].head()

In [ ]:
df.shape

In [ ]:
# Removing rows with missing values in 'CustomerID' and 'Description' columns
df = df.dropna(subset=['CustomerID', 'Description'])

In [ ]:
# Verifying the removal of missing values
df.isnull().sum()

In [ ]:
df.shape

### Checks for duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
# Getting the number of rows in the dataframe
df.shape

### Checks for noise
Analysis of negative values in Quantity and UnitPrice columns

In [ ]:
df[df['Quantity'] < 0].head(20)

In [ ]:
df[df['Quantity'] < 0].shape

There are quite a lot of these transactions. If we had contact with the owners we would request additional information on these negative values to better understand the data. In the absence of that, some kagglers in the Discussion threads have suggested that the negative values represent returned or cancelled items, both of which seem feasible.

In [ ]:
df[df['Quantity'] < 0]['InvoiceNo'].unique()

In [ ]:
display(df[df['Quantity'] < 0]['InvoiceNo'].head(1000).unique())

Is it that all rows with negative quantity values have an Invoice number that starts with "C"?

In [ ]:
df[(df['InvoiceNo'].str[0] != 'C') & (df['Quantity'] < 0)]['InvoiceNo'].head()

In [ ]:
df[(df['InvoiceNo'].str[0] == 'C') & (df['Quantity'] > 0)]['InvoiceNo'].head()

For all rows where Quantity is less than 0, the invoice number starts with "C". It seems these really are "Cancelled" transactions.

In [ ]:
display(df[df['Quantity'] < 0]['StockCode'].head(1000).unique())

Most Stock Codes are 5 characters in length. Let's take a closer look at the exceptions.

In [ ]:
df[(df['StockCode'].str.len() != 5) & (df['Quantity'] < 0)]['StockCode'].head(50)

In [ ]:
df[(df['StockCode'].str.len() == 6) & (df['Quantity'] < 0)].sort_values(by='StockCode').head(50)

The Stock Codes with the letters seem to be different variations of the same product. Let's verify this is the case with quantities more than 0.

In [ ]:
df[(df['StockCode'].str.len() == 6) & (df['Quantity'] > 0)].sort_values(by='StockCode').head(50)

In [ ]:
df[(df['StockCode'].str.len() < 5) & (df['Quantity'] < 0)]['StockCode'].unique()

In [ ]:
df[(df['StockCode'].str.len() < 5) & (df['Quantity'] > 0)]['StockCode'].unique()

In [ ]:
df[(df['StockCode'].str.len() < 5)]['StockCode'].count()

In [ ]:
df[(df['StockCode'].str.len() > 6)]['StockCode'].count()

There are a very small number of transactions with a Stock Code that is not 5 or 6 characters in length. Due to the small number these can safely be dropped from the data set.

In [ ]:
to_remove = df[df['StockCode'].str.len() < 5]['StockCode'].unique()

In [ ]:
to_remove2 = df[df['StockCode'].str.len() > 6]['StockCode'].unique()

In [ ]:
to_remove = np.concatenate((to_remove, to_remove2))
to_remove = np.unique(to_remove)

In [ ]:
df = df[~df['StockCode'].isin(to_remove)]

In [ ]:
# Getting the number of rows in the dataframe
df.shape[0]

In [ ]:
df[df['UnitPrice'] < 0].head(20)

It seems this was removed when null values were handled.

As we have identified the rows with cancelled transaction, let us create a "Transaction Status" column with "Completed" and "cancelled"

In [ ]:
# Filter out the rows with InvoiceNo starting with "C" and create a new column indicating the transaction status
df['Transaction_Status'] = np.where(df['InvoiceNo'].astype(str).str.startswith('C'), 'Cancelled', 'Completed')

# Analyze the characteristics of these rows (considering the new column)
cancelled_transactions = df[df['Transaction_Status'] == 'Cancelled']
cancelled_transactions.describe().drop('CustomerID', axis=1)

### Let's investigate the transactions with 0 price

In [ ]:
df['UnitPrice'].describe()

In [ ]:
df[df['UnitPrice']==0].describe()[['Quantity']]

These are either discounts/puchases with coupons or errors. Given the small number of these transactions (33), we can proceed to remove these transactions from the dataset.

In [ ]:
# Removing records with a unit price of zero to avoid potential data entry errors
df = df[df['UnitPrice'] > 0]

In [ ]:
# Standardize the text to uppercase to maintain uniformity across the dataset
df['Description'] = df['Description'].str.upper()

In [ ]:
# Calculate the occurrence of each unique description and sort them
description_counts = df['Description'].value_counts()

# Get the top 30 descriptions
top_30_descriptions = description_counts[:30]

# Plotting
plt.figure(figsize=(8,6))
plt.barh(top_30_descriptions.index[::-1], top_30_descriptions.values[::-1], color='#ff6200')

# Adding labels and title
plt.xlabel('Number of Occurrences')
plt.ylabel('Description')
plt.title('Top 30 Most Frequent Descriptions')

# Show the plot
plt.show()

In [ ]:
# Resetting the index of the cleaned dataset
df.reset_index(drop=True, inplace=True)

In [ ]:
# Getting the number of rows in the dataframe
df.shape[0]

## Feature Engineering - RFM

Recency - most recent purchase per customer

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df['InvoiceDay'] = df['InvoiceDate'].dt.date

# Find the most recent purchase date for each customer
customer_data = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['InvoiceDay'].max().reset_index()

# Find the most recent date in the entire dataset
most_recent_date = df[df['Transaction_Status'] == 'Completed']['InvoiceDay'].max()

# Convert InvoiceDay to datetime type before subtraction
customer_data['InvoiceDay'] = pd.to_datetime(customer_data['InvoiceDay'])
most_recent_date = pd.to_datetime(most_recent_date)

# Calculate the number of days since the last purchase for each customer
customer_data['Days_Since_Last_Purchase'] = (most_recent_date - customer_data['InvoiceDay']).dt.days

# Remove the InvoiceDay column
customer_data.drop(columns=['InvoiceDay'], inplace=True)

customer_data.head()

### Frequency

Here we create two features that quantify the frequency of a customer's engagement with the retailer:

- __Total Transactions__: The total number of transactions made by a customer.

    

- __Total Products Purchased__: The total number of products (sum of quantities) purchased by a customer across all transactions. This provides insight into the customer's buying behavior in terms of the volume of products purchased.

In [ ]:
# Calculate the total number of transactions made by each customer
total_transactions = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
total_transactions.rename(columns={'InvoiceNo': 'Total_Transactions'}, inplace=True)

# Calculate the total number of products purchased by each customer
total_products_purchased = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['Quantity'].sum().reset_index()
total_products_purchased.rename(columns={'Quantity': 'Total_Products_Purchased'}, inplace=True)

# Merge the new features into the customer_data dataframe
customer_data = pd.merge(customer_data, total_transactions, on='CustomerID')
customer_data = pd.merge(customer_data, total_products_purchased, on='CustomerID')

# Display the first few rows of the customer_data dataframe
customer_data.head()

### Monetary

Here, we create two features that represent the monetary aspect of customer's transactions:

- __Total Spend__: Total amount of money spent per customer, calculated as the sum of the product of `UnitPrice` and `Quantity` for all transactions per customer.

    
- __Average Transaction Value__: __Total Spend__ divided by the __Total Transactions__ per customer. This indicates the average transaction value per customer. This metric is useful in understanding the spending behavior of customers per transaction.

In [ ]:
# Calculate the total spend by each customer
df['Total_Spend'] = df['UnitPrice'] * df['Quantity']
total_spend = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['Total_Spend'].sum().reset_index()

# Calculate the average transaction value for each customer
average_transaction_value = total_spend.merge(total_transactions, on='CustomerID')
average_transaction_value['Average_Transaction_Value'] = average_transaction_value['Total_Spend'] / average_transaction_value['Total_Transactions']

# Merge the new features into the customer_data dataframe
customer_data = pd.merge(customer_data, total_spend, on='CustomerID')
customer_data = pd.merge(customer_data, average_transaction_value[['CustomerID', 'Average_Transaction_Value']], on='CustomerID')

# Display the first few rows of the customer_data dataframe
customer_data.head()

### Other useful features

__Unique Products Purchased__: This feature represents the number of distinct products bought by a customer. A higher value indicates that the customer has a diverse taste or preference, buying a wide range of products, while a lower value might indicate a focused or specific preference. Understanding the diversity in product purchases can help in segmenting customers based on their buying diversity, which can be a critical input in personalizing product recommendations.

In [ ]:
# Calculate the number of unique products purchased by each customer
unique_products_purchased = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['StockCode'].nunique().reset_index()
unique_products_purchased.rename(columns={'StockCode': 'Unique_Products_Purchased'}, inplace=True)

# Merge the new feature into the customer_data dataframe
customer_data = pd.merge(customer_data, unique_products_purchased, on='CustomerID')

# Display the first few rows of the customer_data dataframe
customer_data.head()

__Average Days Between Purchases__: This feature represents the average number of days a customer waits before making another purchase. Understanding this can help in predicting when the customer is likely to make their next purchase, which can be a crucial metric for targeted marketing and personalized promotions.

    
__Favorite Shopping Day__: This denotes the day of the week when the customer shops the most. This information can help in identifying the preferred shopping days of different customer segments, which can be used to optimize marketing strategies and promotions for different days of the week.

    
__Favorite Shopping Hour__: This refers to the hour of the day when the customer shops the most. Identifying the favorite shopping hour can aid in optimizing the timing of marketing campaigns and promotions to align with the times when different customer segments are most active.

    
By including these behavioral features in our dataset, we can create a more rounded view of our customers, which will potentially enhance the effectiveness of the clustering algorithm, leading to more meaningful customer segments.

In [ ]:
# Extract day of week and hour from InvoiceDate
df['Day_Of_Week'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour

# Calculate the average number of days between consecutive purchases
days_between_purchases = df[df['Transaction_Status'] == 'Completed'].groupby('CustomerID')['InvoiceDay'].apply(lambda x: (x.diff().dropna()).apply(lambda y: y.days))
average_days_between_purchases = days_between_purchases.groupby('CustomerID').mean().reset_index()
average_days_between_purchases.rename(columns={'InvoiceDay': 'Average_Days_Between_Purchases'}, inplace=True)

# Find the favorite shopping day of the week
favorite_shopping_day = df[df['Transaction_Status'] == 'Completed'].groupby(['CustomerID', 'Day_Of_Week']).size().reset_index(name='Count')
favorite_shopping_day = favorite_shopping_day.loc[favorite_shopping_day.groupby('CustomerID')['Count'].idxmax()][['CustomerID', 'Day_Of_Week']]

# Find the favorite shopping hour of the day
favorite_shopping_hour = df[df['Transaction_Status'] == 'Completed'].groupby(['CustomerID', 'Hour']).size().reset_index(name='Count')
favorite_shopping_hour = favorite_shopping_hour.loc[favorite_shopping_hour.groupby('CustomerID')['Count'].idxmax()][['CustomerID', 'Hour']]

# Merge the new features into the customer_data dataframe
customer_data = pd.merge(customer_data, average_days_between_purchases, on='CustomerID')
customer_data = pd.merge(customer_data, favorite_shopping_day, on='CustomerID')
customer_data = pd.merge(customer_data, favorite_shopping_hour, on='CustomerID')

# Display the first few rows of the customer_data dataframe
customer_data.head()

__Country__: This feature identifies the country where each customer is located. Different regions might have varying preferences and purchasing behaviors which can be critical in personalizing marketing strategies and inventory planning.

In [ ]:
df['Country'].value_counts(normalize=True).head()

Given that a substantial portion (__89%__) of transactions originate from the __United Kingdom__, we can create a binary feature indicating whether the transaction is from the UK or not. This approach can potentially streamline the clustering process without losing critical geographical information.

In [ ]:
# Group by CustomerID and Country to get the number of transactions per country for each customer
customer_country = df[df['Transaction_Status'] == 'Completed'].groupby(['CustomerID', 'Country']).size().reset_index(name='Number_of_Transactions')

# Get the country with the maximum number of transactions for each customer (in case a customer has transactions from multiple countries)
customer_main_country = customer_country.sort_values('Number_of_Transactions', ascending=False).drop_duplicates('CustomerID')

# Create a binary column indicating whether the customer is from the UK or not
customer_main_country['Is_UK'] = np.where(customer_main_country['Country'] == 'United Kingdom', 1, 0)

# Merge this data with our customer_data dataframe
customer_data = pd.merge(customer_data, customer_main_country[['CustomerID', 'Is_UK']], on='CustomerID', how='left')

# Display the first few rows of the customer_data dataframe
customer_data.head()

In [ ]:
# Display feature distribution
customer_data['Is_UK'].value_counts()

Cancellations

__Cancellation Frequency__: Total number of transactions a customer has canceled. Understanding the frequency of cancellations can help us identify customers who are more likely to cancel transactions.

    
__Cancellation Rate__: Proportion of transactions that a customer has canceled out of all their transactions. This metric gives a normalized view of cancellation behavior.

In [ ]:
# Calculate the total number of transactions made by each customer
total_transactions = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()

# Calculate the number of cancelled transactions for each customer
cancelled_transactions = df[df['Transaction_Status'] == 'Cancelled']
cancellation_frequency = cancelled_transactions.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
cancellation_frequency.rename(columns={'InvoiceNo': 'Cancellation_Frequency'}, inplace=True)

# Merge the Cancellation Frequency data into the customer_data dataframe
customer_data = pd.merge(customer_data, cancellation_frequency, on='CustomerID', how='left')

# Replace NaN values with 0 (for customers who have not cancelled any transaction)
customer_data['Cancellation_Frequency'].fillna(0, inplace=True)

# Calculate the Cancellation Rate
customer_data['Cancellation_Rate'] = customer_data['Cancellation_Frequency'] / total_transactions['InvoiceNo']

# Display the first few rows of the customer_data dataframe
customer_data.head()

__Monthly_Spending_Mean__: Average monthly spend per customer.

In [ ]:
# Extract month and year from InvoiceDate
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month

# Calculate monthly spending for each customer
monthly_spending = df.groupby(['CustomerID', 'Year', 'Month'])['Total_Spend'].sum().reset_index()

# Calculate Seasonal Buying Patterns: We are using monthly frequency as a proxy for seasonal buying patterns
seasonal_buying_patterns = monthly_spending.groupby('CustomerID')['Total_Spend'].agg(['mean']).reset_index()
seasonal_buying_patterns.rename(columns={'mean': 'Monthly_Spending_Mean'}, inplace=True)

# Replace NaN values in Monthly_Spending_Std with 0, implying no variability for customers with single transaction month

# Merge the new features into the customer_data dataframe
customer_data = pd.merge(customer_data, seasonal_buying_patterns, on='CustomerID')

# Display the first few rows of the customer_data dataframe
customer_data.head()

In [ ]:
# Changing the data type of 'CustomerID' to string as it is a unique identifier and not used in mathematical operations
customer_data['CustomerID'] = customer_data['CustomerID'].astype(str)

# Convert data types of columns to optimal types
customer_data = customer_data.convert_dtypes()

In [ ]:
customer_data.head(10)

In [ ]:
customer_data.info()

## Outlier Detection and Treatment

In [ ]:
# prompt: do boxplots of customer_data

customer_data.boxplot(figsize=(16, 10))
plt.xticks(rotation=90)
plt.show()


Given the multi-dimensional nature of the data, we will use the __Isolation Forest__ algorithm that can detect outliers in multi-dimensional spaces.

In [ ]:
# Initializing the IsolationForest model with a contamination parameter of 0.05
model = IsolationForest(contamination=0.05, random_state=0)

# Fitting the model on our dataset (converting DataFrame to NumPy to avoid warning)
customer_data['Outlier_Scores'] = model.fit_predict(customer_data.iloc[:, 1:].to_numpy())

# Creating a new column to identify outliers (1 for inliers and -1 for outliers)
customer_data['Is_Outlier'] = [1 if x == -1 else 0 for x in customer_data['Outlier_Scores']]

# Display the first few rows of the customer_data dataframe
customer_data.head()

<div style="border-radius:10px; padding: 15px; background-color: #ffeacc; font-size:120%; text-align:left">
    
After applying the Isolation Forest algorithm, we have identified the outliers and marked them in a new column named `Is_Outlier`. We have also calculated the outlier scores which represent the anomaly score of each record.

Now let's visualize the distribution of these scores and the number of inliers and outliers detected by the model:

In [ ]:
# Calculate the percentage of inliers and outliers
outlier_percentage = customer_data['Is_Outlier'].value_counts(normalize=True) * 100

# Plotting the percentage of inliers and outliers
plt.figure(figsize=(12, 4))
outlier_percentage.plot(kind='barh', color='#ff6200')

# Adding the percentage labels on the bars
for index, value in enumerate(outlier_percentage):
    plt.text(value, index, f'{value:.2f}%', fontsize=15)

plt.title('Percentage of Inliers and Outliers')
plt.xticks(ticks=np.arange(0, 115, 5))
plt.xlabel('Percentage (%)')
plt.ylabel('Is Outlier')
plt.gca().invert_yaxis()
plt.show()

From the above plot, we can observe that about 5% of the customers have been identified as outliers in our dataset. We now separate the outliers from the main dataset.

In [ ]:
# Separate the outliers for analysis
outliers_data = customer_data[customer_data['Is_Outlier'] == 1]

# Remove the outliers from the main dataset
customer_data_cleaned = customer_data[customer_data['Is_Outlier'] == 0]

# Drop the 'Outlier_Scores' and 'Is_Outlier' columns
customer_data_cleaned = customer_data_cleaned.drop(columns=['Outlier_Scores', 'Is_Outlier'])

# Reset the index of the cleaned data
customer_data_cleaned.reset_index(drop=True, inplace=True)

In [ ]:
# prompt: do boxplots of customer_data

customer_data_cleaned.boxplot(figsize=(16, 10))
plt.xticks(rotation=90)
plt.show()


There are still some outliers observed as per the boxplots, however the most extreme outliers were identified and removed.

In [ ]:
# Getting the number of rows in the cleaned customer dataset
customer_data_cleaned.shape[0]

### Correlation Analysis


In [ ]:
# Find columns with correlation of 0.7 or higher
correlation_matrix = customer_data_cleaned.drop(columns=['CustomerID']).corr()
high_correlation_cols_pos = correlation_matrix[correlation_matrix >= 0.7].stack().drop_duplicates().reset_index()
high_correlation_cols_pos = high_correlation_cols_pos[high_correlation_cols_pos['level_0'] != high_correlation_cols_pos['level_1']]
# Print the columns with high correlation
print(high_correlation_cols_pos)

print("\n")

high_correlation_cols_neg = correlation_matrix[correlation_matrix <= -0.7].stack().drop_duplicates().reset_index()
high_correlation_cols_neg = high_correlation_cols_neg[high_correlation_cols_neg['level_0'] != high_correlation_cols_neg['level_1']]
# Print the columns with high correlation
print(high_correlation_cols_neg)

Variables with high correlations that indicate multicollinearity:

- `Monthly_Spending_Mean` and `Average_Transaction_Value`
    
    
- `Total_Spend` and `Total_Products_Purchased`

    
- `Total_Transactions` and `Total_Spend`
    
    
- `Cancellation_Rate` and `Cancellation_Frequency`
    
    
- `Total_Transactions` and `Total_Products_Purchased`

We can consider treating the multicollinearity through dimensionality reduction techniques such as PCA to create a set of uncorrelated variables.

### Feature Scaling & Dimensionality Reduction

In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()

# List of columns that don't need to be scaled
columns_to_exclude = ['CustomerID', 'Is_UK', 'Day_Of_Week']

# List of columns that need to be scaled
columns_to_scale = customer_data_cleaned.columns.difference(columns_to_exclude)

# Copy the cleaned dataset
customer_data_scaled = customer_data_cleaned.copy()

# Applying the scaler to the necessary columns in the dataset
customer_data_scaled[columns_to_scale] = scaler.fit_transform(customer_data_scaled[columns_to_scale])

# Display the first few rows of the scaled data
customer_data_scaled.head()

Given the multicollinearity we identified in our dataset, we will use PCA for dimensionality reduction as it works well in capturing linear relationships in the data.

In [ ]:
# Setting CustomerID as the index column
customer_data_scaled.set_index('CustomerID', inplace=True)

# Apply PCA
pca = PCA().fit(customer_data_scaled)

# Calculate the Cumulative Sum of the Explained Variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Set the optimal k value (for now we start with 6)
optimal_k = 6

# Set seaborn plot style
sns.set(rc={'axes.facecolor': '#fcf0dc'}, style='darkgrid')

# Plot the cumulative explained variance against the number of components
plt.figure(figsize=(20, 10))

# Bar chart for the explained variance of each component
barplot = sns.barplot(x=list(range(1, len(cumulative_explained_variance) + 1)),
                      y=explained_variance_ratio,
                      color='#fcc36d',
                      alpha=0.8)

# Line plot for the cumulative explained variance
lineplot, = plt.plot(range(0, len(cumulative_explained_variance)), cumulative_explained_variance,
                     marker='o', linestyle='--', color='#ff6200', linewidth=2)

# Plot optimal k value line
optimal_k_line = plt.axvline(optimal_k - 1, color='red', linestyle='--', label=f'Optimal k value = {optimal_k}')

# Set labels and title
plt.xlabel('Number of Components', fontsize=14)
plt.ylabel('Explained Variance', fontsize=14)
plt.title('Cumulative Variance vs. Number of Components', fontsize=18)

# Customize ticks and legend
plt.xticks(range(0, len(cumulative_explained_variance)))
plt.legend(handles=[barplot.patches[0], lineplot, optimal_k_line],
           labels=['Explained Variance of Each Component', 'Cumulative Explained Variance', f'Optimal k value = {optimal_k}'],
           loc=(0.62, 0.1),
           frameon=True,
           framealpha=1.0,
           edgecolor='#ff6200')

# Display the variance values for both graphs on the plots
x_offset = -0.3
y_offset = 0.01
for i, (ev_ratio, cum_ev_ratio) in enumerate(zip(explained_variance_ratio, cumulative_explained_variance)):
    plt.text(i, ev_ratio, f"{ev_ratio:.2f}", ha="center", va="bottom", fontsize=10)
    if i > 0:
        plt.text(i + x_offset, cum_ev_ratio + y_offset, f"{cum_ev_ratio:.2f}", ha="center", va="bottom", fontsize=10)

plt.grid(axis='both')
plt.show()

Here, we can observe that:

- The first component explains approximately 29% of the variance.

- The first two components together explain about 53% of the variance.

- The first three components explain approximately 65% of the variance, and so on.

From the plot, we can see that the increase in cumulative variance starts to slow down after the __5th component__ (which __captures about 81% of the total variance__). Retaining __the first 5 components__ could be a balanced choice, as they together explain a substantial portion of the total variance while reducing the dimensionality of the dataset.

In [ ]:
# Creating a PCA object with 5 components
pca = PCA(n_components=5)

# Fitting and transforming the original data to the new PCA dataframe
customer_data_pca = pca.fit_transform(customer_data_scaled)

# Creating a new dataframe from the PCA dataframe, with columns labeled PC1, PC2, etc.
customer_data_pca = pd.DataFrame(customer_data_pca, columns=['PC'+str(i+1) for i in range(pca.n_components_)])

# Adding the CustomerID index back to the new PCA dataframe
customer_data_pca.index = customer_data_scaled.index

In [ ]:
# Displaying the resulting dataframe based on the PCs
customer_data_pca.head()

## K-Means Clustering

In [ ]:
# Set plot style, and background color
sns.set(style='darkgrid', rc={'axes.facecolor': '#fcf0dc'})

# Set the color palette for the plot
sns.set_palette(['#ff6200'])

# Instantiate the clustering model with the specified parameters
km = KMeans(init='k-means++', n_init=10, max_iter=100, random_state=0)

# Create a figure and axis with the desired size
fig, ax = plt.subplots(figsize=(12, 5))

# Instantiate the KElbowVisualizer with the model and range of k values, and disable the timing plot
visualizer = KElbowVisualizer(km, k=(2, 15), timings=False, ax=ax)

# Fit the data to the visualizer
visualizer.fit(customer_data_pca)

# Finalize and render the figure
visualizer.show();

Using the YellowBrick library for the Elbow method, we observe that the suggested optimal k value is __5__. However, __we don't have a very distinct elbow point in this case__, and it seems that the inertia continues to decrease significantly up to k=5, indicating that __the optimum value of k could be between 3 and 7__. To choose the best k within this range, we employ the __silhouette analysis__.

In [ ]:
def silhouette_analysis(df, start_k, stop_k, figsize=(15, 16)):

    # Set the size of the figure
    plt.figure(figsize=figsize)

    # Create a grid with (stop_k - start_k + 1) rows and 2 columns
    grid = gridspec.GridSpec(stop_k - start_k + 1, 2)

    # Assign the first plot to the first row and both columns
    first_plot = plt.subplot(grid[0, :])

    # First plot: Silhouette scores for different k values
    sns.set_palette(['darkorange'])

    silhouette_scores = []

    # Iterate through the range of k values
    for k in range(start_k, stop_k + 1):
        km = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=100, random_state=0)
        km.fit(df)
        labels = km.predict(df)
        score = silhouette_score(df, labels)
        silhouette_scores.append(score)

    best_k = start_k + silhouette_scores.index(max(silhouette_scores))

    plt.plot(range(start_k, stop_k + 1), silhouette_scores, marker='o')
    plt.xticks(range(start_k, stop_k + 1))
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette score')
    plt.title('Average Silhouette Score for Different k Values', fontsize=15)


    # Add the optimal k value text to the plot
    optimal_k_text = f'The k value with the highest Silhouette score is: {best_k}'
    plt.text(10, 0.23, optimal_k_text, fontsize=12, verticalalignment='bottom',
             horizontalalignment='left', bbox=dict(facecolor='#fcc36d', edgecolor='#ff6200', boxstyle='round, pad=0.5'))


    # Second plot (subplot): Silhouette plots for each k value
    colors = sns.color_palette("bright")

    for i in range(start_k, stop_k + 1):
        km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=0)
        row_idx, col_idx = divmod(i - start_k, 2)

        # Assign the plots to the second, third, and fourth rows
        ax = plt.subplot(grid[row_idx + 1, col_idx])

        visualizer = SilhouetteVisualizer(km, colors=colors, ax=ax)
        visualizer.fit(df)

        # Add the Silhouette score text to the plot
        score = silhouette_score(df, km.labels_)
        ax.text(0.97, 0.02, f'Silhouette Score: {score:.2f}', fontsize=12, \
                ha='right', transform=ax.transAxes, color='red')

        ax.set_title(f'Silhouette Plot for {i} Clusters', fontsize=15)

    plt.tight_layout()
    plt.show()

In [ ]:
silhouette_analysis(customer_data_pca, 3, 12, figsize=(20, 50))

k values of 3 and 4 are very close in silhouette scores. Based on the silhouette plots, k value of 3 seems to be the better choice.

We did the K-Means clustering for both k=3 and k=4, and found that the vizualization and scores(Silhouette Score, Calinski Harabasz Score and
Davies Bouldin Score) were better with value of k=3. In this notebook we only show the K-Means with k=3.

In [ ]:
# Apply KMeans clustering using the optimal k
kmeans = KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=100, random_state=0)
kmeans.fit(customer_data_pca)
customer_data_cleaned['cluster'] = kmeans.labels_
customer_data_pca['cluster'] = kmeans.labels_

In [ ]:
# Display the first few rows of the original dataframe
customer_data_cleaned.head()

In [ ]:
# Compute number of customers
num_observations = len(customer_data_pca)

# Separate the features and the cluster labels
X = customer_data_pca.drop('cluster', axis=1)
clusters = customer_data_pca['cluster']

# Compute the metrics
sil_score = silhouette_score(X, clusters)
calinski_score = calinski_harabasz_score(X, clusters)
davies_score = davies_bouldin_score(X, clusters)

# Create a table to display the metrics and the number of observations
table_data = [
    ["Number of Observations", num_observations],
    ["Silhouette Score", sil_score],
    ["Calinski Harabasz Score", calinski_score],
    ["Davies Bouldin Score", davies_score]
]

# Print the table
print(tabulate(table_data, headers=["Metric", "Value"], tablefmt='pretty'))

In [ ]:
colors = sns.color_palette("bright")

# Calculate the percentage of customers in each cluster
cluster_percentage = (customer_data_pca['cluster'].value_counts(normalize=True) * 100).reset_index()
cluster_percentage.columns = ['Cluster', 'Percentage']
cluster_percentage.sort_values(by='Cluster', inplace=True)

# Create a horizontal bar plot
plt.figure(figsize=(10, 4))
sns.barplot(x='Percentage', y='Cluster', data=cluster_percentage, orient='h', palette=colors)

# Adding percentages on the bars
for index, value in enumerate(cluster_percentage['Percentage']):
    plt.text(value+0.5, index, f'{value:.2f}%')

plt.title('Distribution of Customers Across Clusters', fontsize=14)
plt.xticks(ticks=np.arange(0, 50, 5))
plt.xlabel('Percentage (%)')

# Show the plot
plt.show()

We will use the top 3 PCAs (which capture the most variance in the data) to create a 3D visualization.

In [ ]:
# Setting up the color scheme for the clusters (RGB order)
colors = ['#e8000b', '#1ac938', '#023eff']

In [ ]:
# Create separate data frames for each cluster
cluster_0 = customer_data_pca[customer_data_pca['cluster'] == 0]
cluster_1 = customer_data_pca[customer_data_pca['cluster'] == 1]
cluster_2 = customer_data_pca[customer_data_pca['cluster'] == 2]

# Create a 3D scatter plot
fig = go.Figure()

# Add data points for each cluster separately and specify the color
fig.add_trace(go.Scatter3d(x=cluster_0['PC1'], y=cluster_0['PC2'], z=cluster_0['PC3'],
                           mode='markers', marker=dict(color=colors[0], size=5, opacity=0.4), name='Cluster 0'))
fig.add_trace(go.Scatter3d(x=cluster_1['PC1'], y=cluster_1['PC2'], z=cluster_1['PC3'],
                           mode='markers', marker=dict(color=colors[1], size=5, opacity=0.4), name='Cluster 1'))
fig.add_trace(go.Scatter3d(x=cluster_2['PC1'], y=cluster_2['PC2'], z=cluster_2['PC3'],
                           mode='markers', marker=dict(color=colors[2], size=5, opacity=0.4), name='Cluster 2'))

# Set the title and layout details
fig.update_layout(
    title=dict(text='3D Visualization of Customer Clusters in PCA Space', x=0.5),
    scene=dict(
        xaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC1'),
        yaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC2'),
        zaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC3'),
    ),
    width=900,
    height=800
)

# Show the plot
fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'customer_data_pca' is your DataFrame with columns 'PC1' to 'PC3', 'cluster'

# Selecting the first six principal components
pcs = ['PC1', 'PC2', 'PC3']

# Create a pair grid of scatterplots
sns.set(style="ticks")
sns.pairplot(customer_data_pca, vars=pcs, hue='cluster', palette='viridis', diag_kind='kde', markers='o')
plt.suptitle('Pairwise Scatterplots for Principal Components with Cluster Colors')
plt.show()


## Visualization

We will visualize the clusters in a variety of ways:
- separate radar plots
- integrated radar plot
- line plots

In [ ]:
# Setting 'CustomerID' column as index and assigning it to a new dataframe
df_customer = customer_data_cleaned.set_index('CustomerID')

# Standardize the data (excluding the cluster column)
scaler = StandardScaler()
df_customer_standardized = scaler.fit_transform(df_customer.drop(columns=['cluster'], axis=1))

# Create a new dataframe with standardized values and add the cluster column back
df_customer_standardized = pd.DataFrame(df_customer_standardized, columns=df_customer.columns[:-1], index=df_customer.index)
df_customer_standardized['cluster'] = df_customer['cluster']

# Calculate the centroids of each cluster
cluster_centroids = df_customer_standardized.groupby('cluster').mean()

# Function to create a radar chart
def create_radar_chart(ax, angles, data, color, cluster):
    # Plot the data and fill the area
    ax.fill(angles, data, color=color, alpha=0.4)
    ax.plot(angles, data, color=color, linewidth=2, linestyle='solid')

    # Add a title
    ax.set_title(f'Cluster {cluster}', size=20, color=color, y=1.1)

# Set data
labels=np.array(cluster_centroids.columns)
num_vars = len(labels)

# Compute angle of each axis
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is circular, so we need to "complete the loop" and append the start to the end
labels = np.concatenate((labels, [labels[0]]))
angles += angles[:1]

# Initialize the figure
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(polar=True), nrows=1, ncols=3)


# Create radar chart for each cluster
for i, color in enumerate(colors):
    data = cluster_centroids.loc[i].tolist()
    data += data[:1]  # Complete the loop
    create_radar_chart(ax[i], angles, data, color, i)

# Add input data
ax[0].set_xticks(angles[:-1])
ax[0].set_xticklabels(labels[:-1])

ax[1].set_xticks(angles[:-1])
ax[1].set_xticklabels(labels[:-1])

ax[2].set_xticks(angles[:-1])
ax[2].set_xticklabels(labels[:-1])

# Add a grid
ax[0].grid(color='grey', linewidth=0.5)

# Display the plot
plt.tight_layout()
plt.show()

# Initialize the figure
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(polar=True), nrows=1, ncols=1)

# Create radar chart for each cluster
for i, color in enumerate(colors):
    data = cluster_centroids.loc[i].tolist()
    data += data[:1]  # Complete the loop
    create_radar_chart(ax, angles, data, color, i)

# Add input data
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels[:-1])

# Add a grid
ax.grid(color='grey', linewidth=0.5)

# Display the plot
ax.set_title('All Clusters', size=20, color=color, y=1.1)
plt.tight_layout()
plt.show()



#df_customer_standardized['cluster'] = km_model.labels_

cluster_0_mean = df_customer_standardized[df_customer_standardized['cluster'] == 0].drop('cluster', axis=1).mean()
cluster_1_mean = df_customer_standardized[df_customer_standardized['cluster'] == 1].drop('cluster', axis=1).mean()
cluster_2_mean = df_customer_standardized[df_customer_standardized['cluster'] == 2].drop('cluster', axis=1).mean()

plt.figure(figsize = (30,10) )
plt.plot(cluster_0_mean, label='Cluster 0', linestyle='-', linewidth=3)
plt.plot(cluster_1_mean, label='Cluster 1', linestyle='--', linewidth=3)
plt.plot(cluster_2_mean, label='Cluster 2', linestyle=':', linewidth=3)
plt.legend(fontsize=20)
plt.xticks(fontsize=15, rotation=45)
plt.yticks(fontsize=15)
plt.show()

### Cluster 0

Profile: __Frequent High-Spenders with a High Rate of Cancellations__
    
- Customers in this cluster are high spenders with a very high total spend, and they purchase a wide variety of unique products. These are very likely the wholesale customers.
- They engage in frequent transactions, but also have a high cancellation frequency and rate.  
- These customers have a very low average time between purchases, and they tend to shop early in the day (low `Hour` value).  

____
    
### Cluster 1

Profile: __Weekday Shoppers__  
    
- Customers in this cluster show a moderate level of spending, but their transactions are not very frequent, as indicated by the high `Days_Since_Last_Purchase` and `Average_Days_Between_Purchases`.  
- These customers do most of their shopping throughout the week.

____
    

### Cluster 2

Profile: __Weekend Shoppers__  

These customers are similar to those in Cluster 1, with the below differences:
- They have a greater tendency to shop during the weekends, as indicated by the very high `Day_of_Week` value.  
- These customers prefer shopping late in the day, as indicated by the high `Hour` value.
- These customers purchase a wider variety of products as compared to Cluster 1 customers, but with less frequent purchases than Cluster 1 customers.


We can now add the clusters to the original dataset (with outliers removed). This will enable us to do analysis on transactions per Customer Segment.

In [ ]:
# Step 1: Extract the CustomerIDs of the outliers and remove their transactions from the main dataframe
outlier_customer_ids = outliers_data['CustomerID'].astype('float').unique()
df_filtered = df[~df['CustomerID'].isin(outlier_customer_ids)]

# Step 2: Ensure consistent data type for CustomerID across both dataframes before merging
customer_data_cleaned['CustomerID'] = customer_data_cleaned['CustomerID'].astype('float')

# Step 3: Merge the transaction data with the customer data to get the cluster information for each transaction
merged_data = df_filtered.merge(customer_data_cleaned[['CustomerID', 'cluster']], on='CustomerID', how='inner')

merged_data.head()

In [ ]:
# prompt: horizontal bar chart of top 5 purchases for each cluster

for i in merged_data['cluster'].unique():
  top_5_purchases = merged_data[merged_data['cluster'] == i].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(5)

  top_5_purchases = top_5_purchases.reset_index()

  top_5_purchases.head()

  # Create a horizontal bar chart
  plt.figure(figsize=(4, 2))
  plt.barh(top_5_purchases['Description'], top_5_purchases['Quantity'], color='blue')
  plt.xlabel('Quantity')
  plt.ylabel('Product ID')
  plt.title('Top 5 Purchases for Cluster ' + str(i))
  plt.show()


In [ ]:
result_0 = merged_data[merged_data['cluster'] == 0].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(1)
# Extract the Description from the result
top_cluster0_purchase = result_0.index.get_level_values('Description').values[0]

result_1 = merged_data[merged_data['cluster'] == 1].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(1)
# Extract the Description from the result
top_cluster1_purchase = result_1.index.get_level_values('Description').values[0]

result_2 = merged_data[merged_data['cluster'] == 2].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(1)
# Extract the Description from the result
top_cluster2_purchase = result_2.index.get_level_values('Description').values[0]

In [ ]:
invNo_0 = merged_data[(merged_data['cluster'] == 0) & (merged_data['Description'] == top_cluster0_purchase)]['InvoiceNo']
invNo_1 = merged_data[(merged_data['cluster'] == 1) & (merged_data['Description'] == top_cluster1_purchase)]['InvoiceNo']
invNo_2 = merged_data[(merged_data['cluster'] == 2) & (merged_data['Description'] == top_cluster2_purchase)]['InvoiceNo']

In [ ]:
next_likely_3_purchases_0 = merged_data[(merged_data['InvoiceNo'].isin(invNo_0)) & (merged_data['Description'] != top_cluster0_purchase)].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(3)
next_likely_3_purchases_0 = next_likely_3_purchases_0.reset_index()
# Create a horizontal bar chart
plt.figure(figsize=(4, 2))
plt.barh(next_likely_3_purchases_0['Description'], next_likely_3_purchases_0['Quantity'], color='blue')
plt.xlabel('Quantity')
plt.ylabel('Product ID')
plt.title('Likely purchases for those who bought Cluster 0 top product')


next_likely_3_purchases_1 = merged_data[(merged_data['InvoiceNo'].isin(invNo_1)) & (merged_data['Description'] != top_cluster1_purchase)].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(3)
next_likely_3_purchases_1 = next_likely_3_purchases_1.reset_index()
# Create a horizontal bar chart
plt.figure(figsize=(4, 2))
plt.barh(next_likely_3_purchases_1['Description'], next_likely_3_purchases_1['Quantity'], color='blue')
plt.xlabel('Quantity')
plt.ylabel('Product ID')
plt.title('Likely purchases for those who bought Cluster 1 top product')


next_likely_3_purchases_2 = merged_data[(merged_data['InvoiceNo'].isin(invNo_2)) & (merged_data['Description'] != top_cluster2_purchase)].groupby(['cluster', 'Description'])['Quantity'].sum().nlargest(3)
next_likely_3_purchases_2 = next_likely_3_purchases_2.reset_index()
# Create a horizontal bar chart
plt.figure(figsize=(4, 2))
plt.barh(next_likely_3_purchases_2['Description'], next_likely_3_purchases_2['Quantity'], color='blue')
plt.xlabel('Quantity')
plt.ylabel('Product ID')
plt.title('Likely purchases for those who bought Cluster 2 top product')


plt.show()
